In [ ]:
!pip install pillow

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_hub as hub
from PIL import Image
from tensorflow.keras.preprocessing import image
import ast

# Set up paths
train_data = "data_set/train_data"
val_data = "data_set/val_data"
num_classes = 10
img_height, img_width = 299, 299
batch_size = 32
epochs = 10

# Load the InceptionV3 model with pre-trained weights on ImageNet
base_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet', input_shape=(img_height, img_width, 3))
base_model.trainable = False

# Add custom classification head
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
prediction_layer = tf.keras.layers.Dense(num_classes, activation='softmax')(global_average_layer)
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define data augmentation
train_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Load data and apply data augmentation
train_generator = train_data_gen.flow_from_directory(
    train_data,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_data_gen.flow_from_directory(
    val_data,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Train the model
trained_model = model.fit(train_generator, epochs=epochs, validation_data=val_generator)

# Save the model
model.save('model.h5')

In [ ]:
# Load the model
loaded_model = tf.keras.models.load_model("model.h5")

# Classify a random image
#img = "data_set/test/test1.jpg"
#img = "data_set/test/test2.jpg" 
#img = "data_set/test/test3.jpg" 
#img = "data_set/test/test4.jpg" 
#img = "data_set/test/test5.jpg"
#img = "data_set/test/test6.jpg" 
#img = "data_set/test/test7.jpg" 
#img = "data_set/test/test8.jpg" 
#img = "data_set/test/test9.jpg" 
img = "data_set/test/test10.jpg" 

img = image.load_img(img, target_size=(299, 299))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255
predictions = loaded_model.predict(img_array)

# Get the class name and confidence score for the top prediction
class_names = ["keyboard", "keys", "laptop", "magnifying glass", "mouse", "phone", "router", "satellite dish", "server rack", "usb stick"] # Class names
top_prediction = np.argmax(predictions[0])
class_name = class_names[top_prediction]
confidence_score = predictions[0][top_prediction] * 100
print(f"Class: {class_name}, Confidence: {confidence_score:.2f}%")

Model 2 - Image Classification
base_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet', input_shape=(img_height, img_width, 3)) - loads the InceptionV3 model with pre-trained weights on ImageNet.
include_top = False - to exclude the fully connected layer at the top of the network (further replaced with a custom classification head)
input_shape - specifies the shape of the input images, 299x299 RGB
base_model.trainable = False - prevents the pre-trained weights from being updated during training
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output) - creates a GlobalAveragePooling2D layer (takes the output feature maps of the previous layer and computes the average of each feature map, resulting in a single output value per feature map) and applies it to the output of base_model -> reduces the spatial dimensions of the output and makes it easier to feed into a fully connected layer.
prediction_layer = tf.keras.layers.Dense(num_classes, activation='softmax')(global_average_layer) - creates a fully connected layer (dense layer) with a softmax activation function (ensures that the output of the layer is a probability distribution over the classes), and applies it to the output of the previous global_average_layer.
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer) - creates a new model that takes the input of the pre-trained base_model and outputs the prediction_layer.
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) - compiles the Keras model defined previously
optimizer='adam' - specifies the optimization algorithm that will be used to minimize the loss function during training. adam optimizer: stochastic gradient descent (SGD) optimizer that adapts its learning rate over time.
loss='categorical_crossentropy - specifies the loss function that the optimizer will try to minimize during training. Categorical_crossentropy: used for multi-class classification problems. Loss function: measure how well the model's predictions match the true labels of the training data.
metrics=['accuracy'] - specifies one or more metrics that will be used to evaluate the performance of the model during training and testing.
train_data_gen - performs various image transformations to increase the size of the training set and improve the model's ability to generalize to new data
val_data_gen - rescales the validation data
train_generator - creates a generator object using the ImageDataGenerator class from TensorFlow and The flow_from_directory() method loads images from train_data and applies the data augmentation techniques specified in the train_data_gen
target_size - specifies the size to which images are resized
batch_size - specifies the number of images that will be passed through the network in each batch
class_mode - specifies that the labels will be returned as one-hot encoded categorical arrays
val_generator - same logic
history = model.fit(train_generator, epochs=epochs, validation_data=val_generator) - trains the model using the generator objects train_generator and val_generator
epochs - specifies the number of times the entire training dataset will be passed through the network during training
np.expand_dims(img_array, axis=0) / 255 - adds an extra dimension to the array -> prepare the image for input to a neural network model np.argmax(predictions[0]) - finds the index of the highest prediction score in the predictions array